# Day 11

# 문서 전처리

모든 데이터 분석 모형은 숫자로 구성된 고정 차원 벡터를 독립 변수로 하고 있으므로 문서(document)를 분석을 하는 경우에도 숫자로 구성된 특징 벡터(feature vector)를 문서로부터 추출하는 과정이 필요하다. 이러한 과정을 문서 전처리(document preprocessing)라고 한다.

## BOW (Bag of Words)

문서를 숫자 벡터로 변환하는 가장 기본적인 방법은 BOW (Bag of Words) 이다. BOW 방법에서는 전체 문서 $\{D_1, D_2, \ldots, D_n\}$ 를 구성하는 고정된 단어장(vocabulary) $\{W_1, W_2, \ldots, W_m\}$ 를  만들고 $D_i$라는 개별 문서에 단어장에 해당하는 단어들이 포함되어 있는지를 표시하는 방법이다.

$$ \text{ 만약 단어 } W_j \text{가 문서} D_i \text{ 안에 있으면 }, \;\; \rightarrow x_{ij} = 1 $$ 

## Scikit-Learn 의 문서 전처리 기능

Scikit-Learn 의 feature_extraction.text 서브 패키지는 다음과 같은 문서 전처리용 클래스를 제공한다.

* [`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html): 
 * 문서 집합으로부터 단어의 수를 세어 카운트 행렬을 만든다.
* [`TfidfVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html): 
 * 문서 집합으로부터 단어의 수를 세고 TF-IDF 방식으로 단어의 가중치를 조정한 카운트 행렬을 만든다.
* [`HashingVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html): 
 * hashing trick 을 사용하여 빠르게 카운트 행렬을 만든다.
 

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?',    
]
vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_

{'and': 0,
 'document': 1,
 'first': 2,
 'is': 3,
 'last': 4,
 'one': 5,
 'second': 6,
 'the': 7,
 'third': 8,
 'this': 9}

In [32]:
pd.Series(vect.vocabulary_)

and         0
document    1
first       2
is          3
last        4
one         5
second      6
the         7
third       8
this        9
dtype: int64

In [33]:
vect.transform(['This is the second document.']).toarray()

array([[0, 1, 0, 1, 0, 0, 1, 1, 0, 1]], dtype=int64)

In [34]:
vect.transform(['Something completely new.']).toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [35]:
vect.transform(corpus).toarray()

array([[0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]], dtype=int64)

## 문서 처리 옵션

`CountVectorizer`는 다양한 인수를 가진다. 그 중 중요한 것들은 다음과 같다.

* `stop_words` : 문자열 {‘english’}, 리스트 또는 None (디폴트)
 * stop words 목록.‘english’이면 영어용 스탑 워드 사용.
* `analyzer` : 문자열 {‘word’, ‘char’, ‘char_wb’} 또는 함수
 * 단어 n-그램, 문자 n-그램, 단어 내의 문자 n-그램 
* `tokenizer` : 함수 또는 None (디폴트)
 * 토큰 생성 함수 .
* `token_pattern` : string
 * 토큰 정의용 정규 표현식 
* `ngram_range` : (min_n, max_n) 튜플
 * n-그램 범위 
* `max_df` : 정수 또는 [0.0, 1.0] 사이의 실수. 디폴트 1
 * 단어장에 포함되기 위한 최대 빈도
* `min_df` : 정수 또는 [0.0, 1.0] 사이의 실수.  디폴트 1
 * 단어장에 포함되기 위한 최소 빈도 
* `vocabulary` : 사전이나 리스트
 * 단어장

## Stop Words

Stop Words 는 문서에서 단어장을 생성할 때 무시할 수 있는 단어를 말한다. 보통 영어의 관사나 접속사, 한국어의 조사 등이 여기에 해당한다. `stop_words` 인수로 조절할 수 있다.

In [36]:
vect = CountVectorizer(stop_words=["and", "is", "the", "this"]).fit(corpus)
vect.vocabulary_

{'document': 0, 'first': 1, 'last': 2, 'one': 3, 'second': 4, 'third': 5}

In [37]:
vect = CountVectorizer(stop_words="english").fit(corpus)
vect.vocabulary_

{'document': 0, 'second': 1}

## 토큰(token)

토큰은 문서에서 단어장을 생성할 때 하나의 단어가 되는 단위를 말한다. `analyzer`, `tokenizer`, `token_pattern` 등의 인수로 조절할 수 있다.

In [38]:
vect = CountVectorizer(analyzer="char").fit(corpus)
vect.vocabulary_

{' ': 0,
 '.': 1,
 '?': 2,
 'a': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'h': 8,
 'i': 9,
 'l': 10,
 'm': 11,
 'n': 12,
 'o': 13,
 'r': 14,
 's': 15,
 't': 16,
 'u': 17}

In [39]:
vect = CountVectorizer(token_pattern="t\w+").fit(corpus)
vect.vocabulary_

{'the': 0, 'third': 1, 'this': 2}

In [40]:
import nltk
nltk.download("punkt")
vect = CountVectorizer(tokenizer=nltk.word_tokenize).fit(corpus)
vect.vocabulary_

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jihoon_Kim\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


{'.': 0,
 '?': 1,
 'and': 2,
 'document': 3,
 'first': 4,
 'is': 5,
 'last': 6,
 'one': 7,
 'second': 8,
 'the': 9,
 'third': 10,
 'this': 11}

## n-그램

n-그램은 단어장 생성에 사용할 토큰의 크기를 결정한다. 1-그램은 토큰 하나만 단어로 사용하며 2-그램은 두 개의 연결된 토큰을 하나의 단어로 사용한다.

In [41]:
vect = CountVectorizer(ngram_range=(2,2)).fit(corpus)
vect.vocabulary_

{'and the': 0,
 'first document': 1,
 'is the': 2,
 'is this': 3,
 'last document': 4,
 'second document': 5,
 'second second': 6,
 'the first': 7,
 'the last': 8,
 'the second': 9,
 'the third': 10,
 'third one': 11,
 'this is': 12,
 'this the': 13}

In [42]:
vect = CountVectorizer(ngram_range=(1,2), token_pattern="t\w+").fit(corpus)
vect.vocabulary_

{'the': 0, 'the third': 1, 'third': 2, 'this': 3, 'this the': 4}

## 빈도수

`max_df`, `min_df` 인수를 사용하여 문서에서 토큰이 나타난 횟수를 기준으로 단어장을 구성할 수도 있다. 토큰의 빈도가 `max_df`로 지정한 값을 초과 하거나 `min_df`로 지정한 값보다 작은 경우에는 무시한다. 인수 값은 정수인 경우 횟수, 부동소수점인 경우 비중을 뜻한다. 

In [43]:
vect = CountVectorizer(max_df=4, min_df=2).fit(corpus)
vect.vocabulary_, vect.stop_words_

({'document': 0, 'first': 1, 'is': 2, 'this': 3},
 {'and', 'last', 'one', 'second', 'the', 'third'})

In [44]:
vect.transform(corpus).toarray().sum(axis=0)

array([4, 2, 3, 3], dtype=int64)

## TF-IDF

TF-IDF(Term Frequency – Inverse Document Frequency) 인코딩은 단어를 갯수 그대로 카운트하지 않고 모든 문서에 공통적으로 들어있는 단어의 경우 문서 구별 능력이 떨어진다고 보아 가중치를 축소하는 방법이다. 


구제적으로는 문서 $d$(document)와 단어 $t$ 에 대해 다음과 같이 계산한다.

$$ \text{tf-idf}(d, t) = \text{tf}(d, t) \cdot \text{idf}(t) $$


여기에서

* $\text{tf}(d, t)$: 단어의 빈도수
* $\text{idf}(t)$ : inverse document frequency 
 
 $$ \text{idf}(t) = \log \dfrac{n_d}{1 + \text{df}(t)} $$
 
* $n_d$ : 전체 문서의 수
* $\text{df}(t)$:  단어 $t$를 가진 문서의 수

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
tfidv = TfidfVectorizer().fit(corpus)
tfidv.transform(corpus).toarray()

array([[ 0.        ,  0.38947624,  0.55775063,  0.4629834 ,  0.        ,
         0.        ,  0.        ,  0.32941651,  0.        ,  0.4629834 ],
       [ 0.        ,  0.24151532,  0.        ,  0.28709733,  0.        ,
         0.        ,  0.85737594,  0.20427211,  0.        ,  0.28709733],
       [ 0.55666851,  0.        ,  0.        ,  0.        ,  0.        ,
         0.55666851,  0.        ,  0.26525553,  0.55666851,  0.        ],
       [ 0.        ,  0.38947624,  0.55775063,  0.4629834 ,  0.        ,
         0.        ,  0.        ,  0.32941651,  0.        ,  0.4629834 ],
       [ 0.        ,  0.45333103,  0.        ,  0.        ,  0.80465933,
         0.        ,  0.        ,  0.38342448,  0.        ,  0.        ]])

## Hashing Trick

`CountVectorizer`는 모든 작업을 메모리 상에서 수행하므로 처리할 문서의 크기가 커지면 속도가 느려지거나 실행이 불가능해진다. 이 때  `HashingVectorizer`를 사용하면 해시 함수를 사용하여 단어에 대한 인덱스 번호를 생성하기 때문에 메모리 및 실행 시간을 줄일 수 있다.

In [47]:
from sklearn.datasets import fetch_20newsgroups
twenty = fetch_20newsgroups()
len(twenty.data)

11314

In [48]:
%time CountVectorizer().fit(twenty.data).transform(twenty.data)

Wall time: 5.92 s


<11314x130107 sparse matrix of type '<class 'numpy.int64'>'
	with 1787565 stored elements in Compressed Sparse Row format>

In [49]:
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=10)

In [50]:
%time hv.transform(twenty.data)

Wall time: 3.08 s


<11314x10 sparse matrix of type '<class 'numpy.float64'>'
	with 112863 stored elements in Compressed Sparse Row format>

## 형태소 분석기 이용

In [51]:
corpus = ["imaging", "image", "imagination", "imagine", "buys", "buying", "bought"]
vect = CountVectorizer().fit(corpus)
vect.vocabulary_

{'bought': 0,
 'buying': 1,
 'buys': 2,
 'image': 3,
 'imagination': 4,
 'imagine': 5,
 'imaging': 6}

In [52]:
from sklearn.datasets import fetch_20newsgroups
twenty = fetch_20newsgroups()
docs = twenty.data[:100]

In [53]:
vect = CountVectorizer(stop_words="english", token_pattern="wri\w+").fit(docs)
vect.vocabulary_

{'write': 0,
 'writer': 1,
 'writers': 2,
 'writes': 3,
 'writing': 4,
 'writing_': 5,
 'written': 6}

In [54]:
from nltk.stem import SnowballStemmer

class StemTokenizer(object):
    def __init__(self):
        self.s = SnowballStemmer('english')
        self.t = CountVectorizer(stop_words="english", token_pattern="wri\w+").build_tokenizer()
    def __call__(self, doc):
        return [self.s.stem(t) for t in self.t(doc)]

vect = CountVectorizer(tokenizer=StemTokenizer()).fit(docs)
vect.vocabulary_

{'write': 0, 'writer': 1, 'writing_': 2, 'written': 3}

# Python을 사용한 웹 서비스 개요

Python은 현재 웹 서비스 부분에서 활발히 사용되고 있는 언어의 하나이다. 파이썬을 사용하여 웹서비스를 구축하는 경우, 보통 다음과 같은 구조를 가지게 된다.

<img src="https://datascienceschool.net/upfiles/015a1deab591474a80637365668a8bd6.png" style="width:100%;">

## 리버스 프락시 서버

일반적으로 보안 문제나 정적 파일(static file)의 빠른 서비스, 로드 밸런싱 등을 위해 리버스 프락시(reverse proxy) 서버를 실제 웹 서비스 서버 앞단에 두는 경우가 많다. 리버스 프락시 서버는 클라이언트로부터의 요청을 실제 웹서버나 파일서버로 전달하거나 대리하는 역할을 한다.

많이 쓰이는 리버스 프락시 서버로는 nginx 등이 있다.

## WSGI 웹 어플리케이션 서버

웹 어플리케이션 서버(Web Application Server)는 실제로 클라이언트 호출에 대응하여 HTML 파일 등의 결과물을 출력하는 역할을 하는 서버 프로세스이다. 

클라이언트 호출은 원격에서 이루어지는 함수 호출에 비유할 수 있다. 실제로 함수 호출을 할 수 있는 프로세스는 웹 어플리케이션 서버이지만 함수 자체는 웹 어플리케이션(Web Application)이라고 부르는 일종의 라이브러리와 같은 형태로 구현되어 웹 어플리케이션 서버가 해당 어플리케이션(라이브러리)를 임포트하여 사용한다.

Python의 경우에는 WSGI(Web Server Gateway Interface)이라고 부르는 웹 어플리케이션 규약이 존재하여 모든 웹 어플리케이션은 WSGI 규약에 맞게 구현되어야 한다.

웹 어플리케이션 서버로는 Apache에 modwsgi 모듈을 추가하여 사용할 수 있지만 Python으로 만들어진 웹 어플리케이션을 임포트하기 때문에 웹 어플리케이션 서버도 Python인 경우가 많다. tornado, uwsgi, gnuicorn, werkzeug, twisted 이 많이 사용되고 있다. 

* 웹 어플리케이션 서버의 성능 비교: http://nichol.as/benchmark-of-python-web-servers

## WSGI 웹 어플리케이션

WSGI 웹 어플리케이션은 실제로 다음과 같이 environment(또는 context)와 request 인수를 받아서 response 를 출력하는 함수의 집합이다.

웹 서비스의 각 URL은 이 함수들에 매핑(mapping)되어 웹 어플리케이션 서버가 웹 브라우저의 요청을 받을 때마다 해당 함수를 찾아서 호출하게 된다. 실제 함수 매핑은 웹 어플리케이션 서버보다는 웹 어플리케이션 자체(혹은 웹 어플리케이션 프레임워크)에서 발생하는 경우가 많다.

웹 어플리케이션은 독자적으로 함수 패키지를 만들기 보다는 웹 어플리케이션 프레임워크라고 불리는 구조 및 기반 클래스를 사용하여 만드는 경우가 대부분이다.

## 웹 어플리케이션 프레임워크

웹 어플리케이션 프레임워크는 WSGI 웹 어플리케이션을 빠르고 쉽게 개발하기 위한 기반 구조(archtecture) 및 클래스 라이브러리를 말한다.

Python으로 구현된 다양한 웹 어플리케이션 프레임워크들이 존재한다. 일부 예를 들면 다음과 같다.

* django
* flask
* pyramids
* turbogears
* weppy
* web2py
* bottle
* falcon
* muffin
* pylon
* grokk
* zope



* 웹 어플리케이션 프레임워크 성능 비교: http://klen.github.io/py-frameworks-bench/

웹 어플리케이션의 주 기능 중의 하나는 URL과 함수간의 매핑이다. 예를 들어 django의 경우 다음과 같은 URL 설정을 사용하는데

```python
urlpatterns = [
    url(r'^$', views.index, name='index'),
    url(r'^(?P<question_id>[0-9]+)/$', views.detail, name='detail'),
]
```

이 설정에 따르면 웹 브라우저에서
* http://server/polls/ 를 호출하면 `views` 패키지의 `index` 함수가 호출되고 
* http://server/polls/5/ 를 호출하면 `views` 패키지의 `detail` 함수가 `question_id` 인수 5와 함께 호출된다.

## ORM (Object Relation Mapper)

웹 서비스가 하는 대부분의 일은 데이터 베이스로부터 특정한 데이터를 찾아 보여주는 작업이다. 따라서 데이터 베이스를 쉽게 다울 수 있는 도구가 필수적이다. 

ORM은 데이터 베이스의 구조 즉, 스키마를 Python 클래스 정의로 구현할 수 있도록 한다. 예를 들어 django의 경우 자체적인 ORM을 사용하는데 다음과 같이 클래스를 정의하게 되면

```python
class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')
```

데이터 베이스에서 스키마를 지정하여 테이블을 생성하는 SQL을 자동 생성하여 실행시킨다.

```
CREATE TABLE "question" (
    "id" serial NOT NULL PRIMARY KEY,
    "question_text" varchar(200) NOT NULL,
    "pub_date" timestamp with time zone NOT NULL
);
```





## Template Engine

데이터 베이스로부터 얻어진 데이터는 json이나 txt 같은 단순한 형태로 서비스되기도 하지만 대부분 html 파일을 형태로 서비스된다. 따라서 데이터 요소를 포함하는 html 파일을 동적으로 생성하는 기능을 필요하다. Template Engine은 미리 만들어진 html 파일 template의 일부 문자열을 실제 데이터로 치환(replace)하는 역할을 한다.

django의 경우에는 자체 template engine을 가지고 있지만 flask는 jinja2와 같은 외부 파이썬 패키지를 사용한다. 

예를 들어 jinja2 패키지는 다음과 같은 context와 

```
contries = [
{'Name': 'Afghanistan', 'Population': 22720000},
{'Name': 'Albania', 'Population': 3401200},
{'Name': 'Algeria', 'Population': 31471000},
]
```

다음 template을 합성하여 

```
<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<title>{{ title }}</title>
</head>
<body>
<table>
{% for country in countries %}
<tr><td>{{ country['Name'] }}</td><td>{{ country['Population'] }}</td></tr>
{% endfor %}
</table>
</body>
</html>
```

다음과 같은 html 을 생성할 수 있다.

```
<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<title>Country list</title>
</head>
<body>
<table>
<tr><td>Afghanistan</td><td>22720000</td></tr>
<tr><td>Albania</td><td>3401200</td></tr>
<tr><td>Algeria</td><td>31471000</td></tr>
</table>
</body>
</html>
```